<a href="https://colab.research.google.com/github/IsaiGowthami/CodeClause_Price_Recommendation_for_Online_Sellers/blob/main/Price_Recommendation_for_Online_Sellers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Github Cloning

In [2]:
!git clone https://github.com/IsaiGowthami/CodeClause_Price_Recommendation_for_Online_Sellers.git
%cd CodeClause_Price_Recommendation_for_Online_Sellers

Cloning into 'CodeClause_Price_Recommendation_for_Online_Sellers'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), 35.43 MiB | 6.69 MiB/s, done.
/content/CodeClause_Price_Recommendation_for_Online_Sellers/CodeClause_Price_Recommendation_for_Online_Sellers


Importing Libraries

In [3]:
import numpy as np
import pandas as pd


Reading and verifying the dataset in the input

In [4]:
df=pd.read_csv('/content/price_data.csv',encoding='latin1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom


Collecting the info of input dataset

In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449782 entries, 0 to 449781
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    449782 non-null  object 
 1   StockCode    449781 non-null  object 
 2   Description  448408 non-null  object 
 3   Quantity     449781 non-null  float64
 4   InvoiceDate  449781 non-null  object 
 5   UnitPrice    449781 non-null  float64
 6   CustomerID   338923 non-null  float64
 7   Country      449781 non-null  object 
dtypes: float64(3), object(5)
memory usage: 27.5+ MB


describing the dataset

In [6]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,449781.000000,449781.000000,338923.000000
mean,9.765277,4.692197,15281.391360
std,166.081386,100.523391,1711.891982
min,-74215.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13909.500000
50%,3.000000,2.080000,15154.000000
75%,11.000000,4.130000,16779.000000
max,74215.000000,38970.000000,18287.000000


In [7]:
df.describe(include=object)

,InvoiceNo,StockCode,Description,InvoiceDate,Country
count,449782,449781,448408,449781,449781
unique,22236,4026,4143,20058,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,10/31/2011 14:41,United Kingdom
freq,1114,2037,2090,1114,409753


Evalating and treating the missing value from dataset for efficent output

In [8]:
df.isnull().sum()


InvoiceNo           0
StockCode           1
Description      1374
Quantity            1
InvoiceDate         1
UnitPrice           1
CustomerID     110859
Country             1
dtype: int64

In [9]:
df = df.loc[df['Quantity'] > 0]
df = df.loc[df['UnitPrice'] > 0]

In [10]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     108273
Country             0
dtype: int64

In [11]:
df.loc[df['CustomerID'].isna()].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1.0,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2.0,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4.0,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2.0,12/1/2010 14:32,1.66,NaN,United Kingdom
1447,536544,21790,VINTAGE SNAP CARDS,9.0,12/1/2010 14:32,1.66,NaN,United Kingdom


In [12]:
df.nunique()


InvoiceNo      17010
StockCode       3877
Description     3973
Quantity         344
InvoiceDate    15838
UnitPrice       1202
CustomerID      4063
Country           38
dtype: int64

In [13]:
df.shape

(439354, 8)

In [14]:
df = df.dropna(subset=['CustomerID'])

In [15]:
df.shape

(331081, 8)

In [16]:
df.isnull().sum()


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

# **COLLABORATIVE FILTERING**

The models created by collaborative filtering techniques are based on the prior actions of a user (things previously chosen or purchased, and/or numerical ratings given to those items), as well as comparable choices made by other users. Then, this model is used to forecast the ratings for things or items themselves that the user could be interested in.

In [17]:
customer_item_matrix = df.pivot_table(index='CustomerID', columns='StockCode', values='Quantity',aggfunc='sum')
customer_item_matrix.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214R,90214S,90214V,90214Y,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,7.0


In [18]:
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
customer_item_matrix.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214R,90214S,90214V,90214Y,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12352.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [19]:
customer_item_matrix.shape


(4063, 3624)

# (A) Creating User-to-User Similarity Matrix

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

user_user_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix))
user_user_sim_matrix

,0,1,2,3,4,5,6,7,8,9,...,4053,4054,4055,4056,4057,4058,4059,4060,4061,4062
0,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.104828,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,0.0,1.000000,0.063960,0.048507,0.039057,0.0,0.026261,0.138675,0.096152,0.061159,...,0.052414,0.0,0.053452,0.00000,0.033333,0.063246,0.000000,0.0,0.113333,0.013019
2,0.0,0.063960,1.000000,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,0.000000,...,0.022350,0.0,0.113961,0.00000,0.000000,0.000000,0.000000,0.0,0.127920,0.083269
3,0.0,0.048507,0.051709,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,0.021190,...,0.025425,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.048507,0.000000
4,0.0,0.039057,0.027756,0.031575,1.000000,0.0,0.102568,0.036108,0.089414,0.068248,...,0.150123,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.086793,0.033898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4058,0.0,0.063246,0.000000,0.000000,0.000000,0.0,0.041523,0.000000,0.000000,0.055258,...,0.000000,0.0,0.000000,0.00000,0.105409,1.000000,0.119523,0.0,0.000000,0.000000
4059,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.049629,0.000000,0.000000,0.066046,...,0.000000,0.0,0.000000,0.00000,0.000000,0.119523,1.000000,0.0,0.050395,0.000000
4060,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.039621,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000
4061,0.0,0.113333,0.127920,0.048507,0.086793,0.0,0.113799,0.036980,0.091574,0.081546,...,0.174714,0.0,0.017817,0.04714,0.000000,0.000000,0.050395,0.0,1.000000,0.095472


In [21]:
user_user_sim_matrix.shape

(4063, 4063)

In [22]:
user_user_sim_matrix.columns = customer_item_matrix.index

user_user_sim_matrix['CustomerID'] = customer_item_matrix.index

user_user_sim_matrix = user_user_sim_matrix.set_index('CustomerID')
user_user_sim_matrix.head()

CustomerID,12346.0,12347.0,12348.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,12357.0,...,18272.0,18273.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.104828,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
12347.0,0.0,1.000000,0.063960,0.048507,0.039057,0.0,0.026261,0.138675,0.096152,0.061159,...,0.052414,0.0,0.053452,0.0,0.033333,0.063246,0.0,0.0,0.113333,0.013019
12348.0,0.0,0.063960,1.000000,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,0.000000,...,0.022350,0.0,0.113961,0.0,0.000000,0.000000,0.0,0.0,0.127920,0.083269
12350.0,0.0,0.048507,0.051709,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,0.021190,...,0.025425,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.048507,0.000000
12352.0,0.0,0.039057,0.027756,0.031575,1.000000,0.0,0.102568,0.036108,0.089414,0.068248,...,0.150123,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.086793,0.033898


In [23]:
user_user_sim_matrix.loc[12557].sort_values(ascending=False)

CustomerID
12557.0    1.000000
15970.0    0.544949
17083.0    0.527046
12939.0    0.507093
12732.0    0.447214
             ...   
14618.0    0.000000
14619.0    0.000000
14620.0    0.000000
14622.0    0.000000
18287.0    0.000000
Name: 12557.0, Length: 4063, dtype: float64

In [24]:
items_bought_by_12557 = set(customer_item_matrix.loc[12557].iloc[customer_item_matrix.loc[12557].to_numpy().nonzero()].index)
items_bought_by_12557

{'20725',
 '20727',
 '20728',
 '21238',
 '21239',
 '21240',
 '21242',
 '21243',
 '21244',
 '22383',
 '22384',
 '84997A',
 '84997B',
 '84997C',
 '84997D'}

In [25]:
items_bought_by_12431 = set(customer_item_matrix.loc[12431.0].iloc[customer_item_matrix.loc[12431.0].to_numpy().nonzero()].index)
items_bought_by_12431

{'15056BL',
 '16169E',
 '20675',
 '20677',
 '20685',
 '20712',
 '20719',
 '20724',
 '20725',
 '20726',
 '21000',
 '21001',
 '21080',
 '21094',
 '21115',
 '21164',
 '21217',
 '21239',
 '21240',
 '21242',
 '21243',
 '21244',
 '21245',
 '21428',
 '21481',
 '21507',
 '21524',
 '21527',
 '21533',
 '21537',
 '21555',
 '21622',
 '21706',
 '21707',
 '21731',
 '21745',
 '21770',
 '21791',
 '21880',
 '21883',
 '21931',
 '21933',
 '21935',
 '21936',
 '21937',
 '22027',
 '22029',
 '22037',
 '22045',
 '22090',
 '22131',
 '22138',
 '22191',
 '22192',
 '22193',
 '22194',
 '22195',
 '22196',
 '22204',
 '22219',
 '22326',
 '22328',
 '22329',
 '22352',
 '22354',
 '22356',
 '22382',
 '22383',
 '22385',
 '22411',
 '22413',
 '22423',
 '22466',
 '22467',
 '22492',
 '22505',
 '22617',
 '22629',
 '22631',
 '22690',
 '22692',
 '22697',
 '22698',
 '22699',
 '22712',
 '22726',
 '22727',
 '22728',
 '22729',
 '22730',
 '22785',
 '22786',
 '22835',
 '22846',
 '22859',
 '22907',
 '22941',
 '22951',
 '22965',
 '22966

In [26]:
items_to_recommend_to_12557 = items_bought_by_12557 - items_bought_by_12431
items_to_recommend_to_12557

{'20727', '20728', '21238', '22384', '84997A', '84997B', '84997C', '84997D'}

In [27]:
df.loc[df['StockCode'].isin(items_to_recommend_to_12557), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode')

,Description
StockCode,
84997B,RED 3 PIECE RETROSPOT CUTLERY SET
84997C,BLUE 3 PIECE POLKADOT CUTLERY SET
22384,LUNCH BAG PINK POLKADOT
20728,LUNCH BAG CARS BLUE
20727,LUNCH BAG BLACK SKULL.
84997D,PINK 3 PIECE POLKADOT CUTLERY SET
84997A,GREEN 3 PIECE POLKADOT CUTLERY SET
21238,RED RETROSPOT CUP
84997C,CHILDRENS CUTLERY POLKADOT BLUE


In [28]:
most_similar_user = user_user_sim_matrix.loc[12557].sort_values(ascending=False).reset_index().iloc[1, 0]
most_similar_user

15970.0

In [29]:
def get_items_to_recommend_cust(cust_a): 
  most_similar_user = user_user_sim_matrix.loc[cust_a].sort_values(ascending=False).reset_index().iloc[1, 0]
  items_bought_by_cust_a = set(customer_item_matrix.loc[cust_a].iloc[customer_item_matrix.loc[cust_a].to_numpy().nonzero()].index)
  items_bought_by_cust_b = set(customer_item_matrix.loc[most_similar_user].iloc[customer_item_matrix.loc[most_similar_user].to_numpy().nonzero()].index)
  items_to_recommend_to_a = items_bought_by_cust_b - items_bought_by_cust_a
  items_description = df.loc[df['StockCode'].isin(items_to_recommend_to_a), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode')
  return items_description

In [30]:
get_items_to_recommend_cust(12557.0)

,Description
StockCode,
22662,LUNCH BAG DOLLY GIRL DESIGN
22382,LUNCH BAG SPACEBOY DESIGN
22551,PLASTERS IN TIN SPACEBOY
85032C,CURIOUS IMAGES GIFT WRAP SET


In [31]:
get_items_to_recommend_cust(12431.0)


,Description
StockCode,
84029G,KNITTED UNION FLAG HOT WATER BOTTLE
22749,FELTCRAFT PRINCESS CHARLOTTE DOLL
22960,JAM MAKING SET WITH JARS
22913,RED COAT RACK PARIS FASHION
22914,BLUE COAT RACK PARIS FASHION
...,...
23388,WOODLAND MINI RUCKSACK
23541,"WALL ART ,PUDDINGS"
23523,WALL ART TREASURE AHOY


# **(B) Creating Ite to Item similarity matrix**

In [32]:
item_item_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix.T))
item_item_sim_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,3614,3615,3616,3617,3618,3619,3620,3621,3622,3623
0,1.000000,0.0,0.096825,0.091287,0.0,0.000000,0.091287,0.062932,0.102899,0.102598,...,0.0,0.0,0.0,0.0,0.0,0.030429,0.0,0.071714,0.0,0.063795
1,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.037268,0.051383,0.056011,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
2,0.096825,0.0,1.000000,0.117851,0.0,0.000000,0.058926,0.060933,0.044281,0.066227,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.077152,0.0,0.011766
3,0.091287,0.0,0.117851,1.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
4,0.000000,0.0,0.000000,0.000000,1.0,0.447214,0.064550,0.044499,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [33]:
item_item_sim_matrix.shape


(3624, 3624)

In [34]:
item_item_sim_matrix.columns = customer_item_matrix.T.index

item_item_sim_matrix['StockCode'] = customer_item_matrix.T.index
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')
item_item_sim_matrix.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214R,90214S,90214V,90214Y,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.0,0.096825,0.091287,0.0,0.000000,0.091287,0.062932,0.102899,0.102598,...,0.0,0.0,0.0,0.0,0.0,0.030429,0.0,0.071714,0.0,0.063795
10080,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.037268,0.051383,0.056011,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
10120,0.096825,0.0,1.000000,0.117851,0.0,0.000000,0.058926,0.060933,0.044281,0.066227,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.077152,0.0,0.011766
10123C,0.091287,0.0,0.117851,1.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
10124A,0.000000,0.0,0.000000,0.000000,1.0,0.447214,0.064550,0.044499,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [35]:
item_item_sim_matrix.loc['10002'].sort_values(ascending=False)


StockCode
10002     1.000000
21826     0.197642
16010     0.182574
90059C    0.182574
90059E    0.182574
            ...   
79144B    0.000000
21414     0.000000
79062D    0.000000
21416     0.000000
16202B    0.000000
Name: 10002, Length: 3624, dtype: float64

In [36]:
top_10_similar_items = list(item_item_sim_matrix.loc['10002'].sort_values(ascending=False).iloc[:10].index)
top_10_similar_items

['10002',
 '21826',
 '16010',
 '90059C',
 '90059E',
 '90059F',
 '90059B',
 '84535A',
 '21086',
 '21439']

In [37]:
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [38]:
df.loc[df['StockCode'] == '90210A']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
28848,538661,90210A,GREY ACRYLIC FACETED BANGLE,12.0,12/13/2010 15:42,1.25,15194.0,United Kingdom
28887,538662,90210A,GREY ACRYLIC FACETED BANGLE,12.0,12/13/2010 15:44,1.25,15159.0,United Kingdom
56707,541110,90210A,GREY ACRYLIC FACETED BANGLE,2.0,1/13/2011 15:11,2.95,15916.0,United Kingdom


In [39]:
df.loc[df['StockCode'] == '90210A'][:1]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
28848,538661,90210A,GREY ACRYLIC FACETED BANGLE,12.0,12/13/2010 15:42,1.25,15194.0,United Kingdom


In [40]:
df.loc[df['StockCode'].isin(top_10_similar_items), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode').loc[top_10_similar_items]

,Description
StockCode,
10002,INFLATABLE POLITICAL GLOBE
21826,EIGHT PIECE DINOSAUR SET
16010,FOLDING CAMPING SCISSOR W/KNIF & S
90059C,DIAMANTE HAIR GRIP PACK/2 MONTANA
90059E,DIAMANTE HAIR GRIP PACK/2 RUBY
90059F,DIAMANTE HAIR GRIP PACK/2 LT ROSE
90059B,DIAMANTE HAIR GRIP PACK/2 BLACK DIA
84535A,ENGLISH ROSE NOTEBOOK A6 SIZE
21086,SET/6 RED SPOTTY PAPER CUPS


In [41]:
def get_top_similar_items(item):
  top_10_similar_items = list(item_item_sim_matrix.loc[item].sort_values(ascending=False).iloc[:10].index)
  top_10 = df.loc[df['StockCode'].isin(top_10_similar_items), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode').loc[top_10_similar_items]
  return top_10

In [42]:
get_top_similar_items('84029E')


,Description
StockCode,
84029E,RED WOOLLY HOTTIE WHITE HEART.
84029G,KNITTED UNION FLAG HOT WATER BOTTLE
21479,WHITE SKULL HOT WATER BOTTLE
21485,RETROSPOT HEART HOT WATER BOTTLE
22111,SCOTTIE DOG HOT WATER BOTTLE
22112,CHOCOLATE HOT WATER BOTTLE
22114,HOT WATER BOTTLE TEA AND SYMPATHY
84030E,ENGLISH ROSE HOT WATER BOTTLE
21481,FAWN BLUE HOT WATER BOTTLE
